In [18]:
# Before we start, remember to activate your conda environment.
## Write 'conda create --name myenv numpy pandas scipy matplotlib' into your terminal
## Then activate the environment: 'conda activate myenv'
## Remember to check that your Python interpreter is correct

# First step: Import the libraries we're going to use (PANDAS, NUMPY, MATPLOTLIB)
import pandas as pd
import numpy as np
import matplotlib as plt
import os 

In [19]:
# Second step: We're going to read our Resources.csv and test to see if we can see it.
SNP_file = pd.read_csv('Resources/Template.csv')    #You should always open the Data_Extraction_Code 
                                                       #folder to have a relative path similar to this one
SNP_file.head()

,DOI,GENE,SNP ID,P-VALUE (GWAS)


In [20]:
# Third step: Now let's create a second dataframe (to keep from editing the original)
SNP_editable = SNP_file
SNP_editable.head()

,DOI,GENE,SNP ID,P-VALUE (GWAS)


In [21]:
#Fourth step: Let's merge the dataframe and use SNP ID as our merge.
# Let's read our second .csv file
Our_SNP_List = pd.read_csv('Resources/Input_Files/IA_test.csv')
Our_SNP_List.head()

# And...

,DOI,GENE,SNP ID,P-VALUE (GWAS)
0,10.3340/jkns.2022.0026,BOLL,rs700651,NaN
1,10.3340/jkns.2022.0026,[U]EDNRA,rs6841581,NaN
2,10.3340/jkns.2022.0026,PTCH1,rs1105980,NaN
3,10.3340/jkns.2022.0026,CCDC3,rs12412014,NaN
4,10.3340/jkns.2022.0026,LINC00457,rs1536847,NaN


In [22]:
# MERGE! 
merge_data = pd.merge(Our_SNP_List, SNP_file, on=["SNP ID", "GENE", "DOI", "P-VALUE (GWAS)"], how='left')
merge_data.head(10)

# Now we have our own database set up as a dataframe and filled it with our columns. 
# If you merged two databases, great! If you did not, boo! Let's output the merged_data (if you did not merge two databases,
# then ignore the output.
output_path = 'Resources/OutputFromExtractionCode'
merge_data.to_csv(os.path.join(output_path, 'Merged_data_1.csv'), index=False)


#merge_data.to_csv('Merged_data_1.csv', index=False)

In [23]:
# Now let's drop all the data we collected and just stay with the two things that matter most to us: 
# the SNP ID and the Gene:
SNP_ID_And_Gene_From_Merged_Data = merge_data[['GENE', 'SNP ID']]
SNP_ID_And_Gene_From_Merged_Data.head(10)

,GENE,SNP ID
0,BOLL,rs700651
1,[U]EDNRA,rs6841581
2,PTCH1,rs1105980
3,CCDC3,rs12412014
4,LINC00457,rs1536847
5,C5orf60,rs62405726
6,METTL4,rs549315
7,RGPD4,rs700855
8,RGPD4,rs328025
9,MALL,rs117802391


In [24]:
### We'll be using this for our RENTREZ module in R'. Let's output it twice (one for keeps and one we can modify safely)
SNP_ID_And_Gene_From_Merged_Data.to_csv(os.path.join(output_path, 'Original_Gene_And_dbSNP_ID_Merged.csv'), index=False)
SNP_ID_And_Gene_From_Merged_Data.to_csv(os.path.join(output_path, 'Editable_Gene_And_dbSNP_ID_Merged.csv'), index=False)

In [25]:
### We can already do something pretty cool, which is determine which gene had more pathogenic SNPs.
### We can sort it:
SNP_Sorted_By_Gene =  SNP_ID_And_Gene_From_Merged_Data.sort_values(by='GENE')
SNP_Sorted_By_Gene.head(10)

,GENE,SNP ID
223,8q11,rs10958409
226,9p,rs2891168
224,9p21,rs1333040
225,9p21,rs10757278
127,ADAMTS12,rs1364044
131,ADAMTS13,rs2285489
129,ADAMTS13,rs4962153
128,ADAMTS13,rs739469
130,ADAMTS13,rs2301612
266,ADAMTS15,rs767345140


In [26]:
### And get the unique count for each gene:
Unique_Genes = SNP_Sorted_By_Gene['GENE'].value_counts().reset_index()
Unique_Genes.columns = ['GENE', 'Reported SNPs']
Unique_Genes.head(20)

,GENE,Reported SNPs
0,VCAN,15
1,MMP13,11
2,MMP2,11
3,PCNT,9
4,RNF213,6
5,MMP24,6
6,LIMK1,6
7,MMP26,6
8,COL1A2,5
9,CDKN2B-AS1,5


In [27]:
### And we should group them by gene so we can see which SNPs are actually related:
grouped_SNPs = SNP_Sorted_By_Gene.groupby('GENE')['SNP ID'].apply(list).reset_index()
grouped_SNPs.head(20)

# groupby('category') groups the data by the 'category' column.
# apply(list) applies the list function to each group, resulting in a list of strings for each unique category.
# reset_index() is used to turn the result back into a DataFrame.

,GENE,SNP ID
0,8q11,[rs10958409]
1,9p,[rs2891168]
2,9p21,"[rs1333040, rs10757278]"
3,ADAMTS12,[rs1364044]
4,ADAMTS13,"[rs2285489, rs4962153, rs739469, rs2301612]"
5,ADAMTS15,[rs767345140]
6,ADAMTS2,[rs11750568]
7,AL359922.1,[rs4977574]
8,ANGPTL6,"[rs147149731, rs559282550]"
9,ARHGAP32,[rs371331393]


In [28]:
### Now let's output them as CSV files:
SNP_Sorted_By_Gene.to_csv(os.path.join(output_path, 'SNP_Sorted_By_Gene.csv'), index=False)
Unique_Genes.to_csv(os.path.join(output_path, 'Unique_Genes.csv'), index=False)
grouped_SNPs.to_csv(os.path.join(output_path, 'grouped_SNPs.csv'), index=False)


In [29]:
#### Now let's turn to our python R file